$Setting~Up~Environment$

In [7]:
import pandas as pd
import bs4 
import nltk
from nltk.corpus import stopwords
import numpy as np
from gensim.models.phrases import Phrases, Phraser
from gensim.models import FastText
from gensim.models import Word2Vec
from gensim.test.utils import datapath
import joblib

  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [28 lines of output]
      /Users/alexz/.pyenv/versions/3.11.0/bin/python3.11: No module named pip
      Traceback (most recent call last):
        File "<string>", line 38, in __init__
      ModuleNotFoundError: No module named 'pybind11'
      
      During handling of the above exception, another exception occurred:
      
      Traceback (most recent call last):
        File "/Users/alexz/.pyenv/versions/3.11.0/lib/python3.11/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
        File "/Users/alexz/.pyenv/versions/3.11.0/lib/python3.11/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hoo

$Load~in~dataset$

In [11]:
# Specify the path of the output file
output_file_path = 'full_text_cleaned.txt'

# Open the output file in write mode
with open(output_file_path, 'w') as output_file:
    with open('/Users/alexz/Downloads/full_text_disinfo', 'r') as f:
        for line in f:
            # Process each line here, remove '\n' using strip
            processed_line = line.strip()
            # Write the processed line to the output file
            output_file.write(processed_line + '\n')


with open(output_file_path, 'r') as output_file:
    full_text_clean = output_file.readlines()
    full_text_clean = [text.replace('\n', '') for text in full_text_clean]

with open('/Users/alexz/Sentiment Analysis of COVID-19 Misinformation with FastText/Sentiment-Analysis-of-COVID-19-Misinformation-with-FastText/Sentiment-Analysis-of-COVID-19-Misinformation-with-FastText---Processing-Data-and-Visualizing-in-R_files/full_text_testsites_official', 'r') as test:
    full_text_test = test.readlines()
    full_text_test = [text.replace('\n', '') for text in full_text_test] 
    

$Pre~ Processing:~ FastText$

In [21]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


#Remove stopwords
stop_words = set(stopwords.words('english'))
def remove_stopwords(words_list):
    return [word for word in words_list if word not in stop_words]
remove_stopwords(full_text_clean)

#Lemmatizer
def lemma_stem_text(words_list):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words_list]

lemma_stem_text(full_text_clean)

[nltk_data] Downloading package stopwords to /Users/alexz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/alexz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/alexz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['“There is a general sense of fear, so people look out for information — and YouTube becomes one of the places to look for it.”',
 'BuzzFeed News Reporter',
 'BuzzFeed News Reporter',
 'A screenshot of the most popular YouTube video about the coronavirus.',
 'The most popular YouTube video in the world about the coronavirus was created by a channel called Wonderful Secrets of the World, which typically publishes Hindi-language videos about sports and cars, as well as roundups like “Top 5 Secret Places Hidden in Famous Locations" or “30 Amazing Facts About Human Body." The channel, the logo of which looks uncannily like that of Volkswagen, hides its subscriber count, but 16 of its videos have been viewed over a million times — the most popular being the coronavirus explainer, which has been seen 13.6 million times as of Wednesday.',
 'Wonderful Secrets of the World isn’t an outlier. It is the tip of a huge Indian YouTube iceberg of content about the outbreak. The disease has 75,280 con

$Extract~phrases/sentences~from~title~column~in~dataset$

In [22]:

# Split each row in 'title' only if it is a string
sent = [row.split() if isinstance(row, str) else [] for row in full_text_clean]

#Create phrases object from Phrases method in gensim library
phrases = Phrases(sent, min_count=30, progress_per=10000) #LOOK INTO MIN COUNT AND PROGRESS_PER=10000 
#Create a Phraser object called sentences which contains sentences inside from the original dataframe column "title"
sentences = phrases[sent]

$Initializing,~building~ vocabulary,~and~training~for~fastText~model$

In [23]:
#why no stop-words in FastText????????????????????????????????????????????????????

#Build FastText vocabulary
model_covid_misinfo = FastText(vector_size=100, window=5, min_count=5, workers=4, min_n=1) #size=n has been deprecated
model_covid_misinfo.build_vocab(sentences)
print(len(model_covid_misinfo.wv.key_to_index)) 

6924


In [26]:
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(sentences, test_size=0.2, random_state=42)
#Train FastText model
model_covid_misinfo.train(sentences, total_examples=len(sentences), epochs=10)#look into appropriate length of epochs for 5000 row dataset

(1791416, 2488930)

In [37]:
#Save model for later
path = 'fasttext_covidmisinfo.joblib'
joblib.dump(model_covid_misinfo,path)

['fasttext_covidmisinfo.joblib']

0.4519331

In [69]:
def calculate_similarities(model, pairs):
    results = []
    for word1, word2 in pairs:
        try:
            similarity = model.wv.n_similarity([word1], [word2])
            results.append({'word1': word1, 'word2': word2, 'similarity': similarity})
        except KeyError as e:
            results.append({'word1': word1, 'word2': word2, 'similarity': 'Word not in vocabulary'})
    return pd.DataFrame(results)

if __name__ == '__main__':
    model = model_covid_misinfo  # Load your trained model
    word_pairs = [('vaccine','cure'),
                  ('hydroxychloroquine', 'cure'),
                  ('bleach','cure'),
                  ('marijuana','cure'),
                  ('ivermectin','cure'),
                  ('vaccine','effective'),
                  ('hydroxychloroquine','effective'),
                  ('bleach','effective'),
                  ('marijuana','effective'),
                  ('ivermectin','effective'),
                  ('Jesus', 'effective'),
                  ('prayer', 'effective'),
                  ('microchips','vaccine'),
                  ('Bill Gates','vaccine contains microchips'),
                  ('hydroxychloroquine','good'),
                  ('vaccine','good'),
                  ('hydroxychloroquine','hydroxychloroquine is good'),
                  ('hydroxychloroquine','hydroxychloroquine is bad'),
                  ('vaccine','vaccine is good'),
                  ('vaccine','vaccine is bad'),
                  ('vaccine','vaccine is poison')
                                        ]
    similarity_df = calculate_similarities(model, word_pairs)
    print(similarity_df)    

                 word1                        word2  similarity
0              vaccine                         cure    0.421430
1   hydroxychloroquine                         cure    0.411877
2               bleach                         cure    0.337188
3            marijuana                         cure   -0.052350
4           ivermectin                         cure    0.294740
5              vaccine                    effective    0.639924
6   hydroxychloroquine                    effective    0.193248
7               bleach                    effective    0.196845
8            marijuana                    effective   -0.159917
9           ivermectin                    effective    0.825845
10               Jesus                    effective    0.116451
11              prayer                    effective    0.076502
12          microchips                      vaccine    0.595403
13          Bill Gates  vaccine contains microchips    0.236191
14  hydroxychloroquine                  